# 時系列データセットの準備

このノートブックは、時系列データにノイズを付与してデータセットを準備します。

## 手順

1. Google Driveをマウント
2. 必要なファイルをアップロード
3. データセット準備を実行
4. 結果をGoogle Driveに保存


## 1. Google Driveをマウント


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## 2. 作業ディレクトリの設定


In [ ]:
import os
import sys

# Google Driveにsoturonフォルダをそのままアップロード
project_root = '/content/drive/MyDrive/soturon'  # soturonフォルダのルート
baseline_folder = f'{project_root}/畳み込み'  # 畳み込みフォルダ

if os.path.exists(baseline_folder):
    os.chdir(baseline_folder)
    sys.path.insert(0, baseline_folder)
    sys.path.insert(0, project_root)  # プロジェクトルートも追加
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {baseline_folder} が見つかりません。")
    print("Google Driveにsoturonフォルダを作成して、ファイルをアップロードしてください。")


## 3. 必要なファイルの確認


In [ ]:
# 必要なファイルの存在確認
required_files = [
    f'{project_root}/data_lowF_noise_time.pickle',  # 時系列データ
    f'{project_root}/ノイズの付与(共通)/timeseries_noise.py',
    f'{baseline_folder}/dataset_timeseries.py',
]

missing_files = []
for file in required_files:
    if not os.path.exists(file):
        if file.startswith(baseline_folder):
            missing_files.append(file.replace(baseline_folder + '/', '畳み込み/'))
        elif file.startswith(project_root):
            missing_files.append(file.replace(project_root + '/', ''))

if missing_files:
    print("⚠️ 以下のファイルが見つかりません:")
    for file in missing_files:
        print(f"  - {file}")
    print(f"\n📁 Google Driveの {project_root} にファイルをアップロードしてください。")
else:
    print("✅ すべての必要なファイルが見つかりました！")


## 4. データセット準備の実行


In [ ]:
# データセット準備スクリプトを実行
# スクリプト内でdata_lowF_noise_time.pickleを読み込んで処理します

# まず、データファイルのパスを確認
data_path = f'{project_root}/data_lowF_noise_time.pickle'
if os.path.exists(data_path):
    print(f"✅ 時系列データファイルが見つかりました: {data_path}")
    
    # データセット準備スクリプトをインポートして実行
    # スクリプトを直接実行する方法
    import subprocess
    import sys
    
    # スクリプトのパス
    script_path = f'{baseline_folder}/dataset_timeseries.py'
    
    if os.path.exists(script_path):
        print(f"\nデータセット準備を実行中...")
        print("=" * 60)
        
        # スクリプトを実行
        result = subprocess.run(
            [sys.executable, script_path],
            cwd=baseline_folder,
            capture_output=True,
            text=True
        )
        
        print(result.stdout)
        if result.stderr:
            print("エラー:")
            print(result.stderr)
        
        print("=" * 60)
    else:
        print(f"⚠️ スクリプトが見つかりません: {script_path}")
else:
    print(f"⚠️ 時系列データファイルが見つかりません: {data_path}")
    print("先に convert_psd_to_timeseries.ipynb を実行して、時系列データを生成してください。")


## 5. 結果の確認


In [ ]:
import pickle
import torch
import matplotlib.pyplot as plt
import numpy as np

# 生成されたデータセットを読み込んで確認
dataset_path = f'{baseline_folder}/baseline_dataset_timeseries.pickle'

if os.path.exists(dataset_path):
    print(f"✅ データセットファイルが見つかりました: {dataset_path}")
    
    with open(dataset_path, 'rb') as f:
        dataset = pickle.load(f)
    
    train_data = dataset['train']['data']
    train_labels = dataset['train']['labels']
    val_data = dataset['val']['data']
    val_labels = dataset['val']['labels']
    test_data = dataset['test']['data']
    test_labels = dataset['test']['labels']
    config = dataset['config']
    
    print(f"\nデータセットの情報:")
    print(f"  訓練データ: {len(train_data):,}サンプル")
    print(f"  検証データ: {len(val_data):,}サンプル")
    print(f"  テストデータ: {len(test_data):,}サンプル")
    print(f"  データ形状: {train_data.shape}")
    print(f"\n設定:")
    print(f"  区間数: {config['num_intervals']}")
    print(f"  ノイズタイプ: {config['noise_type']}")
    print(f"  ノイズレベル: {config['noise_level']}")
    print(f"  データタイプ: {config.get('data_type', 'unknown')}")
    
    # サンプルデータを可視化
    sample_idx = 0
    sample = train_data[sample_idx].numpy()
    label = train_labels[sample_idx].item()
    
    points_per_interval = len(sample) // config['num_intervals']
    noise_start = label * points_per_interval
    noise_end = noise_start + points_per_interval
    
    plt.figure(figsize=(14, 5))
    plt.plot(sample, linewidth=1, alpha=0.7, label='ノイズ付与後')
    plt.axvspan(noise_start, noise_end, alpha=0.2, color='red', label=f'ノイズ区間 {label}')
    plt.title(f'時系列データ（サンプル {sample_idx}、ラベル: {label}）')
    plt.xlabel('時間ポイント')
    plt.ylabel('正規化後の値')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nサンプル {sample_idx} の統計情報:")
    print(f"  最小値: {sample.min():.6f}")
    print(f"  最大値: {sample.max():.6f}")
    print(f"  平均値: {sample.mean():.6f}")
    print(f"  標準偏差: {sample.std():.6f}")
    print(f"  ノイズ区間: {noise_start}-{noise_end}")
    
else:
    print(f"⚠️ データセットファイルが見つかりません: {dataset_path}")
